In [2]:
import pandas as pd

file_path = "/Users/gouthamjekkula/Desktop/Gravity_V202211.csv" 
chunksize = 500_000

filtered_chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
    filtered = chunk[
        (chunk['iso3_o'] == 'IRL') & (chunk['iso3_d'] == 'GBR')
    ]
    if not filtered.empty:
        filtered_chunks.append(filtered)

gravity_irl_uk = pd.concat(filtered_chunks)
print(f"Filtered rows: {gravity_irl_uk.shape[0]}")

# Keep country_id columns for traceability
cols_to_keep = [
    'year', 
    'country_id_o', 'country_id_d', 'iso3_o', 'iso3_d',
    'distw_harmonic', 'distw_arithmetic', 'dist',
    'contig', 'comlang_off', 'comcol', 'col45',
    'gdp_o', 'gdp_d', 'gdpcap_o', 'gdpcap_d',
    'pop_o', 'pop_d', 'rta_coverage', 'rta_type'
]

gravity_irl_uk_clean = gravity_irl_uk[cols_to_keep]

# Save filtered file
output_file = "gravity_irl_uk_clean1.xlsx"
gravity_irl_uk_clean.to_excel(output_file, index=False)
print(f"Saved filtered dataset to: {output_file}")


Filtered rows: 74
Saved filtered dataset to: gravity_irl_uk_clean1.xlsx


In [4]:
import pandas as pd
import re
from pathlib import Path

IE_FILE = Path("/Users/gouthamjekkula/Desktop/Thesis Folder/Ireland Current Gdp Values.xlsx")  # Ireland file (Unpivoted sheet)
UK_FILE = Path("/Users/gouthamjekkula/Desktop/Thesis Folder/United Kingdom Current GDP values.xlsx")       # UK file
GBP_TO_EUR = 1.16
OUT_FILE = Path("Ireland_United Kingdom_GDP1.xlsx")
def normalize_period(s):
    """Return 'YYYYQx' (no spaces, uppercase) from variants like '1995Q1', '1995 Q1', etc."""
    s = str(s).strip().upper()
    m = re.match(r"^\s*(\d{4})\s*Q\s*([1-4])\s*$", s.replace("Q", " Q "))
    if m:
        return f"{m.group(1)}Q{m.group(2)}"
    # also allow '1995Q1' directly
    m = re.match(r"^\s*(\d{4})Q([1-4])\s*$", s)
    if m:
        return f"{m.group(1)}Q{m.group(2)}"
    return None

# IRELAND 
ie = pd.read_excel(IE_FILE, sheet_name="Unpivoted")
ie.columns = [str(c).strip() for c in ie.columns]

# Keep the exact series shown in your screenshot
label_col = "Statistic Label" if "Statistic Label" in ie.columns else "STATISTIC LABEL"
val_col   = "VALUE" if "VALUE" in ie.columns else "Value"
q_col     = "Quarter" if "Quarter" in ie.columns else "QUARTER"

ie = ie[ie[label_col].str.strip().eq("GDP at Current Market Prices (Seasonally Adjusted)")].copy()
ie["Period"] = ie[q_col].map(normalize_period)
ie["Ireland GDP values"] = pd.to_numeric(ie[val_col], errors="coerce")
ie = ie[["Period", "Ireland GDP values"]].dropna(subset=["Period", "Ireland GDP values"])
ie["Ireland"] = "Ireland"

# Diagnostics
print(f"Ireland periods: {ie['Period'].nunique()} rows, sample ->", ie['Period'].head(3).tolist())

#UNITED KINGDOM 
# autodetected the header row containing 'Quarterly' and 'GDP Values'
raw = pd.read_excel(UK_FILE, header=None)
header_row = None
for i in range(min(30, len(raw))):  # search first 30 rows
    row_vals = [str(x).strip() for x in raw.iloc[i].tolist()]
    if ("Quarterly" in row_vals) and ("GDP Values" in row_vals):
        header_row = i
        break

if header_row is None:
    # Fallback: from your screenshot it looked like headers start at Excel row 10 -> zero-based 9
    header_row = 9

uk = pd.read_excel(UK_FILE, header=header_row)
uk.columns = [str(c).strip() for c in uk.columns]

# To Make sure expected columns exist; else take first two columns and rename
if not {"Quarterly", "GDP Values"}.issubset(set(uk.columns)):
    uk = uk.iloc[:, :2].copy()
    uk.columns = ["Quarterly", "GDP Values"]

uk["Period"] = uk["Quarterly"].map(normalize_period)
uk["United Kingdom Values"] = pd.to_numeric(uk["GDP Values"], errors="coerce") * GBP_TO_EUR
uk = uk[["Period", "United Kingdom Values"]].dropna(subset=["Period", "United Kingdom Values"])
uk["United Kingdom"] = "United Kingdom"

# Diagnostics
print(f"UK periods: {uk['Period'].nunique()} rows, sample ->", uk['Period'].head(3).tolist())

# ---------- MERGE ----------
combined = (ie.merge(uk, on="Period", how="inner")
              .sort_values("Period")
              .reset_index(drop=True))

# If it comes out empty, show why
if combined.empty:
    ie_set = set(ie["Period"])
    uk_set = set(uk["Period"])
    only_ie = sorted(list(ie_set - uk_set))[:5]
    only_uk = sorted(list(uk_set - ie_set))[:5]
    print("Merged result is empty. Example periods only in Ireland:", only_ie)
    print("Example periods only in UK:", only_uk)

# Final columns
combined["Unit value"] = "Euro million"
combined["Ireland"] = "Ireland"                # ensure present
combined["United Kingdom"] = "United Kingdom"  # ensure present
combined = combined[[
    "Period",
    "Ireland",
    "Ireland GDP values",
    "United Kingdom",
    "United Kingdom Values",
    "Unit value"
]]

# Save
combined.to_excel(OUT_FILE, index=False)
print(f"[OK] Saved → {OUT_FILE}")
print(combined.head(8).to_string(index=False))

Ireland periods: 121 rows, sample -> ['1995Q1', '1995Q2', '1995Q3']
UK periods: 121 rows, sample -> ['1995Q1', '1995Q2', '1995Q3']
[OK] Saved → Ireland_United Kingdom_GDP1.xlsx
Period Ireland  Ireland GDP values United Kingdom  United Kingdom Values   Unit value
1995Q1 Ireland               13334 United Kingdom              242042.12 Euro million
1995Q2 Ireland               13648 United Kingdom              244467.68 Euro million
1995Q3 Ireland               13689 United Kingdom              248954.56 Euro million
1995Q4 Ireland               14053 United Kingdom              252675.84 Euro million
1996Q1 Ireland               14643 United Kingdom              256862.28 Euro million
1996Q2 Ireland               14989 United Kingdom              262376.92 Euro million
1996Q3 Ireland               14920 United Kingdom              266697.92 Euro million
1996Q4 Ireland               15503 United Kingdom              269654.76 Euro million


In [6]:
import pandas as pd
import re
from pathlib import Path

# Input Files
NOMINAL_FILE = Path("/Users/gouthamjekkula/Desktop/Thesis Folder/Ireland_United Kingdom_GDP.xlsx")
IE_DEFLATOR  = Path("/Users/gouthamjekkula/Downloads/alfredgraph.xlsx")
UK_DEFLATOR  = Path("/Users/gouthamjekkula/Downloads/GBRGDPDEFQISMEI.xlsx")
OUT_FILE     = Path("Ireland_United Kingdom_Real_GDP1.xlsx")

def to_period_quarter(dt):
    """Map datetime to YYYYQn."""
    y = dt.year
    q = (dt.month - 1) // 3 + 1
    return f"{y}Q{q}"

def normalize_period_any(x):
    """Convert various formats to YYYYQn."""
    if pd.isna(x):
        return None
    if hasattr(x, "year"):
        return to_period_quarter(x)
    s = str(x).strip().upper()
    m = re.match(r"^(\d{4})\s*Q\s*([1-4])$", s.replace("Q"," Q "))
    if m:
        return f"{m.group(1)}Q{m.group(2)}"
    m = re.match(r"^Q\s*([1-4])\s*(\d{4})$", s)
    if m:
        return f"{m.group(2)}Q{m.group(1)}"
    try:
        dt = pd.to_datetime(s, errors="raise")
        return to_period_quarter(dt)
    except Exception:
        return None

def read_alfred_quarterly_latest(filepath, series_colname):
    """Read ALFRED/FRED quarterly sheet and return Period + latest value."""
    df = pd.read_excel(filepath, sheet_name="Quarterly", header=0)
    date_col = df.columns[0]
    if series_colname not in df.columns:
        raise ValueError(f"Series '{series_colname}' not found in {filepath.name}")
    out = pd.DataFrame({
        "Period": df[date_col].apply(normalize_period_any),
        "value": pd.to_numeric(df[series_colname], errors="coerce")
    }).dropna(subset=["Period","value"])
    return out.sort_values("Period")

#1) Nominal GDP 
nom = pd.read_excel(NOMINAL_FILE)
nom.columns = [str(c).strip() for c in nom.columns]
nom["Period"] = nom["Period"].apply(normalize_period_any)
nom = nom.rename(columns={
    "Ireland GDP values": "IE_nominal_eur_m",
    "United Kingdom Values": "UK_nominal_eur_m"
})[["Period","IE_nominal_eur_m","UK_nominal_eur_m"]]

# 2) Ireland deflator (2015=100)
#Ireland deflator (2015=100) — explicit column 
# Uses the most recent vintage you showed: IRLGDPDEFQISMEI_20231209
ie_raw = pd.read_excel("alfredgraph.xlsx", sheet_name="Quarterly")

# Normalize to YYYYQn 
ie_raw["Period"] = ie_raw["observation_date"].apply(normalize_period_any)

# Pick the exact column; fall back to the rightmost series if the name changes
COL_IRL = "IRLGDPDEFQISMEI_20231209"
if COL_IRL not in ie_raw.columns:
    # fallback: take the last non-date column
    cand_cols = [c for c in ie_raw.columns if c != "observation_date"]
    COL_IRL = cand_cols[-1]

ie_def = (ie_raw[["Period", COL_IRL]]
          .rename(columns={COL_IRL: "IE_deflator_2015=100"})
          .dropna(subset=["Period", "IE_deflator_2015=100"])
          .sort_values("Period")
          .reset_index(drop=True))

#3) UK deflator (2015=100)
uk_def = read_alfred_quarterly_latest(UK_DEFLATOR, series_colname="GBRGDPDEFQISMEI")
uk_def = uk_def.rename(columns={"value": "UK_deflator_2015=100"})

# 4) Merge all
df = (nom.merge(ie_def, on="Period", how="left")
         .merge(uk_def, on="Period", how="left")
         .sort_values("Period")
         .reset_index(drop=True))

# 5) Fill missing deflators (e.g., 2024–2025)
for col in ["IE_deflator_2015=100","UK_deflator_2015=100"]:
    df[col] = df[col].astype(float).ffill().bfill()

# 6) Convert nominal → real (2015 prices, €m) 
df["Ireland (Real, €m, 2015=100)"] = (df["IE_nominal_eur_m"] / df["IE_deflator_2015=100"]) * 100.0
df["United Kingdom (Real, €m, 2015=100)"] = (df["UK_nominal_eur_m"] / df["UK_deflator_2015=100"]) * 100.0

#7) Save output 
out = df[[
    "Period",
    "IE_nominal_eur_m",
    "Ireland (Real, €m, 2015=100)",
    "UK_nominal_eur_m",
    "United Kingdom (Real, €m, 2015=100)"
]].rename(columns={
    "IE_nominal_eur_m": "Ireland (Nominal, €m)",
    "UK_nominal_eur_m": "United Kingdom (Nominal, €m)"
})

out.to_excel(OUT_FILE, index=False)
print(f"[OK] Saved real GDP dataset → {OUT_FILE}")
print(out.head(8).to_string(index=False))


[OK] Saved real GDP dataset → Ireland_United Kingdom_Real_GDP1.xlsx
Period  Ireland (Nominal, €m)  Ireland (Real, €m, 2015=100)  United Kingdom (Nominal, €m)  United Kingdom (Real, €m, 2015=100)
1995Q1                  13334                  22207.465571                     242042.12                        345035.709797
1995Q2                  13648                  22730.425237                     244467.68                        348493.392353
1995Q3                  13689                  22798.709779                     248954.56                        354889.526322
1995Q4                  14053                  23404.943278                     252675.84                        360194.282726
1996Q1                  14643                  24387.574498                     256862.28                        366162.133681
1996Q2                  14989                  24963.829417                     262376.92                        374023.359349
1996Q3                  14920              

In [16]:
import pandas as pd
import numpy as np

# FILE PATHS
GRAVITY_XLSX = "/Users/gouthamjekkula/Desktop/Thesis Folder/gravity_irl_uk_clean1.xlsx"
GDP_XLSX     = "/Users/gouthamjekkula/Desktop/Thesis Folder/Ireland_United Kingdom_Real_GDP1.xlsx"
OUT_XLSX     = "GRAVITY_GDP_IE_UK_2015_2023_MONTHLY.xlsx"


# DATE WINDOW
START = pd.Timestamp("2015-01-01")
END   = pd.Timestamp("2023-12-01")


# 1) LOAD GRAVITY AND EXPAND TO MONTHLY

g = pd.read_excel(GRAVITY_XLSX)

# IRL-GBR only
g = g[(g["iso3_o"].isin(["IRL","GBR"])) & (g["iso3_d"].isin(["IRL","GBR"]))]

# Expand YEAR -> 12 months
def expand_year_row(row):
    y = int(row["year"])
    months = pd.date_range(f"{y}-01-01", periods=12, freq="MS")
    rep = pd.DataFrame({col: row[col] for col in g.columns}, index=months)
    rep.index.name = "Date"
    return rep.reset_index()

g_monthly = pd.concat([expand_year_row(r) for _, r in g.iterrows()], ignore_index=True)

# Filter 2015-2021 (since gravity stops at 2021)
g_monthly["Date"] = pd.to_datetime(g_monthly["Date"])
g_monthly = g_monthly[(g_monthly["Date"] >= START) & (g_monthly["Date"] <= pd.Timestamp("2021-12-01"))]


# 2) CREATE EXTENSION 2022–2023 (using last available row as template)

last_row = g_monthly[g_monthly["year"] == 2021].iloc[0].copy()

months_2022_2023 = pd.date_range("2022-01-01", "2023-12-01", freq="MS")

rows = []
for m in months_2022_2023:
    new_row = last_row.copy()
    new_row["year"] = m.year
    new_row["Date"] = m
    rows.append(new_row)

g_ext = pd.DataFrame(rows)

# Combine with 2015–2021 monthly
g_full = pd.concat([g_monthly, g_ext], ignore_index=True)

# 3) LOAD GDP & UPSAMPLE QUARTERLY → MONTHLY

gdpr = pd.read_excel(GDP_XLSX)

IE_REAL_COL  = "Ireland (Real, €m, 2015=100)"
UK_REAL_COL  = "United Kingdom (Real, €m, 2015=100)"
PERIOD_COL   = "Period"

def period_to_months(p):
    s = str(p).strip().upper().replace(" ", "")
    if "Q" in s and len(s) >= 6:
        year = int(s[:4]); q = int(s[-1])
        start_month = {1:1, 2:4, 3:7, 4:10}[q]
        return pd.date_range(pd.Timestamp(year, start_month, 1), periods=3, freq="MS")
    if "-" in s:
        y, m = map(int, s.split("-")[:2])
        return pd.DatetimeIndex([pd.Timestamp(y, m, 1)])
    if s.isdigit() and len(s) == 4:
        y = int(s)
        return pd.date_range(pd.Timestamp(y, 1, 1), periods=12, freq="MS")
    raise ValueError(f"Unrecognized Period: {p}")

rows = []
for _, r in gdpr.iterrows():
    months = period_to_months(r[PERIOD_COL])
    df = pd.DataFrame({
        "Date": months,
        "GDP_IE_real_eur_m": r[IE_REAL_COL],
        "GDP_UK_real_eur_m": r[UK_REAL_COL]
    })
    rows.append(df)

gdp_m = pd.concat(rows, ignore_index=True)
gdp_m = gdp_m[(gdp_m["Date"] >= START) & (gdp_m["Date"] <= END)]
gdp_m = gdp_m.groupby("Date", as_index=False).last()

# 4) MERGE GRAVITY + GDP
merged = g_full.merge(gdp_m, on="Date", how="left")


# 5) OVERWRITE WITH EXTERNAL DATA (2015–2023 monthly series)
# rta_coverage values (length = 108 months)
rta_coverage_vals = [
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
]

rta_type_vals = [
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
]

# pop_o and pop_d values (108 each)
pop_o_vals = [4701957]*12 + [4762722]*12 + [4827445]*12 + [4898022]*12 + [4976456]*12 + [5039747]*12 + [5110585]*12 + [5212836]*12 + [5307600]*12
pop_d_vals = [65088000]*12 + [65607000]*12 + [65966000]*12 + [66289000]*12 + [66631000]*12 + [66744000]*12 + [66984000]*12 + [67604000]*12 + [68492000]*12

#Assign to merged
merged = merged.sort_values("Date").reset_index(drop=True)

merged["rta_coverage"] = rta_coverage_vals
merged["rta_type"]     = rta_type_vals
merged["pop_o"]        = pop_o_vals
merged["pop_d"]        = pop_d_vals

# 6) SAVE
merged.to_excel(OUT_XLSX, index=False)
print("Final dataset saved:", OUT_XLSX)
print("Rows:", len(merged))
print("Cols:", list(merged.columns))


✅ Final dataset saved: GRAVITY_GDP_IE_UK_2015_2023_MONTHLY.xlsx
Rows: 108
Cols: ['Date', 'year', 'country_id_o', 'country_id_d', 'iso3_o', 'iso3_d', 'distw_harmonic', 'distw_arithmetic', 'dist', 'contig', 'comlang_off', 'comcol', 'col45', 'gdp_o', 'gdp_d', 'gdpcap_o', 'gdpcap_d', 'pop_o', 'pop_d', 'rta_coverage', 'rta_type', 'GDP_IE_real_eur_m', 'GDP_UK_real_eur_m']


In [20]:
import pandas as pd
import numpy as np

INFILE  = "/Users/gouthamjekkula/Desktop/Thesis Folder/GRAVITY_GDP_IE_UK_2015_2023_MONTHLY.xlsx"
OUT_XLS = "PPML_Gravity1.xlsx"

# Load
df = pd.read_excel(INFILE)
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"]).dt.to_period("M").dt.to_timestamp()

#Check columns
expected_cols = ["GDP_IE_real_eur_m", "GDP_UK_real_eur_m", "pop_o", "pop_d"]
missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing column(s): {missing}")

# Clean/guard: make sure populations > 0 to avoid division errors
for c in ["pop_o", "pop_d"]:
    if (df[c] <= 0).any():
        raise ValueError(f"Non‑positive values found in {c}. Fix before proceeding.")

# Calculate GDP per capita (Euro/person)

df["gdpcap_o"] = (df["GDP_IE_real_eur_m"] * 1_000_000) / df["pop_o"]
df["gdpcap_d"] = (df["GDP_UK_real_eur_m"] * 1_000_000) / df["pop_d"]

# Optional: round for readability (keep full precision if you prefer)
df["gdpcap_o"] = df["gdpcap_o"].round(2)
df["gdpcap_d"] = df["gdpcap_d"].round(2)

# Quick diagnostics
stats = df[["gdpcap_o", "gdpcap_d"]].agg(["min", "max", "mean"])
print("\nGDP per capita summary (Euro/person):")
print(stats)

# Flag unrealistic values
for col in ["gdpcap_o", "gdpcap_d"]:
    bad = df[(df[col] < 5_000) | (df[col] > 150_000)]
    if not bad.empty:
        print(f"\n⚠ Warning: {len(bad)} rows in {col} outside expected range [5k, 150k].")
        print(bad[["Date", "iso3_o", "iso3_d", col]].head(10).to_string(index=False))

#Save
df.to_excel(OUT_XLS, index=False)
print(f"\nSaved: {OUT_XLS} ")
print(f"Rows: {len(df):,}  |  Columns: {len(df.columns)}")



GDP per capita summary (Euro/person):
          gdpcap_o   gdpcap_d
min   13993.610000  6995.6800
max   22859.150000  9132.2400
mean  17928.290278  8740.6425

✅ Saved: PPML_Gravity1.xlsx 
Rows: 108  |  Columns: 23


In [24]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

IN_FILE  = Path("/Users/gouthamjekkula/Desktop/Thesis Folder/IRL&UK Tariffs.xlsx")  
OUT_WIDE = Path("Tariffs_2015_2023_Monthly_Sectors_Wide_AHS.xlsx")

START_YEAR = 2015
END_YEAR   = 2023

USD_TO_EUR   = 0.93
KUSD_TO_MEUR = USD_TO_EUR / 1000.0

# Sector mapping tuned to  file’s Product Namme
SECTOR_CONTAINS = {
    "Pharma":       ["pharma"],                     
    "Meat":         ["meat &", "meat and"],        
    "Dairy":        ["dairy"],                     
    "Beverages":    ["beverages"],
    "Agriculture": [
        "live animals except fish",
        "cereals/cereal",                           
        "vegetables and fruit",
        "sugar/sugar",                              
        "coffee/tea/cocoa/spices",
        "animal feed ex unml cer.",
        "misc food products",
    ],
    "Vegetables":   ["crude anim/veg mater nes"],   
}

# mapping to ISO3 for clean joins later
ISO3_MAP = {
    "Ireland": "IRL",
    "United Kingdom": "GBR",
    "UK": "GBR",
    "Great Britain": "GBR",
    
}

def year_to_month_starts(year: int):
    return pd.date_range(f"{year}-01-01", f"{year}-12-01", freq="MS")

def map_sector(name_lower: str):
    for sector, needles in SECTOR_CONTAINS.items():
        for needle in needles:
            if needle in name_lower:
                return sector
    return None

# 1) Read
tar = pd.read_excel(IN_FILE)
tar.columns = [str(c).strip() for c in tar.columns]

need = [
    "Tariff Year", "Product Name", "Reporter Name", "Partner Name",
    "DutyType", "Weighted Average", "Simple Average", "Imports Value in 1000 USD"
]
missing = [c for c in need if c not in tar.columns]
if missing:
    raise ValueError(f"Missing columns in {IN_FILE.name}: {missing}")

# 2) For AHS only (applied rates)
tar = tar[tar["DutyType"].astype(str).str.upper().eq("AHS")].copy()

# 3) Clean types & restrict years (2015–2023)
tar["Tariff Year"] = pd.to_numeric(tar["Tariff Year"], errors="coerce").astype("Int64")
for c in ["Weighted Average", "Simple Average", "Imports Value in 1000 USD"]:
    tar[c] = pd.to_numeric(tar[c], errors="coerce")

tar = tar[tar["Tariff Year"].between(START_YEAR, END_YEAR, inclusive="both")].copy()

# 4) Map to six sectors
plower = tar["Product Name"].astype(str).str.lower()
tar["Sector"] = plower.apply(map_sector)
tar = tar.dropna(subset=["Sector", "Tariff Year"]).copy()

# 5) Aggregate by (Year × Reporter × Partner × Sector) with imports-weighted average
value_cols = ["Weighted Average", "Simple Average", "Imports Value in 1000 USD"]

def agg_sector(g):
    w = g["Imports Value in 1000 USD"].fillna(0)
    x = g["Weighted Average"]
    if (w > 0).any() and x.notna().any():
        weighted_avg = np.average(x.fillna(0), weights=w)
    else:
        weighted_avg = x.mean()
    simple_avg = g["Simple Average"].mean()
    imports_sum_kusd = g["Imports Value in 1000 USD"].sum(min_count=1)
    return pd.Series({
        "WeightedAvg": weighted_avg,
        "SimpleAvg": simple_avg,
        "Imports_EuroM": imports_sum_kusd * KUSD_TO_MEUR
    })

group_cols = ["Tariff Year","Reporter Name","Partner Name","Sector"]
agg = (tar.groupby(group_cols, dropna=False, observed=False)[value_cols]
         .apply(agg_sector)
         .reset_index())

# If nothing matched (unlikely), It write's empty template for 2015–2023 and exit
if agg.empty:
    all_months = pd.date_range(f"{START_YEAR}-01-01", f"{END_YEAR}-12-01", freq="MS")
    cols = ["Date","Reporter Name","Partner Name"]
    for s in ["Pharma","Agriculture","Beverages","Dairy","Meat","Vegetables"]:
        cols += [f"{s}_WeightedAvg", f"{s}_SimpleAvg", f"{s}_Imports_EuroM"]
    out = pd.DataFrame({"Date": all_months})
    for c in cols:
        if c != "Date":
            out[c] = np.nan
    out.to_excel(OUT_WIDE, index=False)
    print(f"[WARN] No AHS rows matched; wrote empty template → {OUT_WIDE}")
    raise SystemExit

# 6) Pivot to wide by sector for each (Year/Reporter/Partner)
w_weighted = (agg.pivot(index=["Tariff Year","Reporter Name","Partner Name"],
                        columns="Sector", values="WeightedAvg")
                .add_suffix("_WeightedAvg"))
w_simple   = (agg.pivot(index=["Tariff Year","Reporter Name","Partner Name"],
                        columns="Sector", values="SimpleAvg")
                .add_suffix("_SimpleAvg"))
w_imports  = (agg.pivot(index=["Tariff Year","Reporter Name","Partner Name"],
                        columns="Sector", values="Imports_EuroM")
                .add_suffix("_Imports_EuroM"))

wide_year = (w_weighted.join(w_simple, how="outer")
                        .join(w_imports, how="outer")
                        .reset_index())

# 7) Ensure all expected sector columns exist
expected_cols = []
for s in ["Pharma","Agriculture","Beverages","Dairy","Meat","Vegetables"]:
    expected_cols += [f"{s}_WeightedAvg", f"{s}_SimpleAvg", f"{s}_Imports_EuroM"]
for c in expected_cols:
    if c not in wide_year.columns:
        wide_year[c] = np.nan

# 8) Expand each year to 12 months - Date
months = pd.DataFrame({"Month": range(1, 13)})
wide_year["Tariff Year"] = wide_year["Tariff Year"].astype(int)
wide_year["key"] = 1; months["key"] = 1
monthly = (wide_year.merge(months, on="key", how="outer").drop(columns="key"))
monthly["Date"] = pd.to_datetime(dict(year=monthly["Tariff Year"], month=monthly["Month"], day=1))
monthly = monthly.drop(columns=["Month"]).sort_values(["Reporter Name","Partner Name","Date"])

# 9) Build full monthly grid 2015–2023 for all reporter/partner pairs seen
pairs = monthly[["Reporter Name","Partner Name"]].drop_duplicates().reset_index(drop=True)
all_months = pd.date_range(f"{START_YEAR}-01-01", f"{END_YEAR}-12-01", freq="MS")
full_grid = (pairs.assign(_k=1)
                  .merge(pd.DataFrame({"_k":1,"Date":all_months}), on="_k", how="outer")
                  .drop(columns="_k"))

# 10) Left‑join observed months (2015–2023) onto full grid
keep_cols = ["Date","Reporter Name","Partner Name","Tariff Year"] + expected_cols
result_wide = (full_grid.merge(monthly[keep_cols], on=["Date","Reporter Name","Partner Name"], how="left")
                        .sort_values(["Reporter Name","Partner Name","Date"])
                        .reset_index(drop=True))



result_long = pd.concat(long_rows, ignore_index=True)
result_long["ReporterISO3"] = result_long["Reporter Name"].map(to_iso3)
result_long["PartnerISO3"]  = result_long["Partner Name"].map(to_iso3)

# 12) SAVE
result_wide.to_excel(OUT_WIDE, index=False)

print(f"[OK] Saved → {OUT_WIDE}")
print(f"Rows (wide): {len(result_wide):,} | Cols (wide): {len(result_wide.columns)}")
print(result_wide.head(6).to_string(index=False))


[OK] Saved → Tariffs_2015_2023_Monthly_Sectors_Wide_AHS.xlsx
Rows (wide): 108 | Cols (wide): 22
 Reporter Name Partner Name       Date  Tariff Year  Pharma_WeightedAvg  Pharma_SimpleAvg  Pharma_Imports_EuroM  Agriculture_WeightedAvg  Agriculture_SimpleAvg  Agriculture_Imports_EuroM  Beverages_WeightedAvg  Beverages_SimpleAvg  Beverages_Imports_EuroM  Dairy_WeightedAvg  Dairy_SimpleAvg  Dairy_Imports_EuroM  Meat_WeightedAvg  Meat_SimpleAvg  Meat_Imports_EuroM  Vegetables_WeightedAvg  Vegetables_SimpleAvg  Vegetables_Imports_EuroM
United Kingdom      Ireland 2015-01-01          NaN                 NaN               NaN                   NaN                      NaN                    NaN                        NaN                    NaN                  NaN                      NaN                NaN              NaN                  NaN               NaN             NaN                 NaN                     NaN                   NaN                       NaN
United Kingdom      Irelan

In [26]:
import pandas as pd

#File paths
PPML_FILE = "/Users/gouthamjekkula/Desktop/Thesis Folder/PPML_Gravity1.xlsx"
AHS_FILE  = "/Users/gouthamjekkula/Desktop/Thesis Folder/Tariffs_2015_2023_Monthly_Sectors_Wide_AHS.xlsx"
OUT_FILE  = "PPML_Gravity_Tariffs1.xlsx"

# 1) Read PPML_Gravity and normalize Date
ppml = pd.read_excel(PPML_FILE)
ppml.columns = [c.strip() for c in ppml.columns]
if "Date" not in ppml.columns:
    raise ValueError("PPML_Gravity.xlsx must contain a 'Date' column.")
ppml["Date"] = pd.to_datetime(ppml["Date"], errors="coerce").dt.to_period("M").dt.to_timestamp()

# 2) Read AHS tariffs and normalize Date
ahs = pd.read_excel(AHS_FILE)
ahs.columns = [c.strip() for c in ahs.columns]
if "Date" not in ahs.columns:
    raise ValueError("Tariffs_2015_2023_Monthly_Sectors_Wide_AHS.xlsx must contain a 'Date' column.")
ahs["Date"] = pd.to_datetime(ahs["Date"], errors="coerce").dt.to_period("M").dt.to_timestamp()

# 3) Identify tariff columns in the AHS file
sector_prefixes = ("Pharma_", "Agriculture_", "Beverages_", "Dairy_", "Meat_", "Vegetables_")
tariff_cols = [c for c in ahs.columns if c.startswith(sector_prefixes)]
keep_ahs = ["Date"] + [c for c in ahs.columns if c in ["Reporter Name","Partner Name"]] + tariff_cols
ahs = ahs[keep_ahs].copy()

# 4) If PPML already has old tariff columns, drop them to avoid duplicates
old_tariff_cols = [c for c in ppml.columns if c.startswith(sector_prefixes)]
if old_tariff_cols:
    ppml = ppml.drop(columns=old_tariff_cols)

# 5) Choose merge keys
merge_keys = ["Date", "Reporter Name", "Partner Name"]
can_merge_on_triplet = all(k in ppml.columns for k in merge_keys) and all(k in ahs.columns for k in merge_keys)

if can_merge_on_triplet:
    # align name fields
    for k in ["Reporter Name","Partner Name"]:
        ppml[k] = ppml[k].astype(str).str.strip()
        ahs[k]  = ahs[k].astype(str).str.strip()
    merged = ppml.merge(ahs, on=merge_keys, how="left")
    print("[INFO] Merged on Date + Reporter Name + Partner Name.")
else:
    merged = ppml.merge(ahs.drop(columns=[c for c in ["Reporter Name","Partner Name"] if c in ahs.columns]),
                        on="Date", how="left")
    print("[INFO] Merged on Date only.")

# 6) Convert tariff columns to numeric and fill missing with 0
if tariff_cols:
    merged[tariff_cols] = merged[tariff_cols].apply(pd.to_numeric, errors="coerce").fillna(0)

# 7) Save
merged.to_excel(OUT_FILE, index=False)
print(f"[OK] Saved → {OUT_FILE}")
print(f"Rows: {len(merged):,} | Columns: {len(merged.columns)}")
print("Tariff columns added:", sorted(tariff_cols)[:10], "…")


[INFO] Merged on Date only.
[OK] Saved → PPML_Gravity_Tariffs1.xlsx
Rows: 108 | Columns: 41
Tariff columns added: ['Agriculture_Imports_EuroM', 'Agriculture_SimpleAvg', 'Agriculture_WeightedAvg', 'Beverages_Imports_EuroM', 'Beverages_SimpleAvg', 'Beverages_WeightedAvg', 'Dairy_Imports_EuroM', 'Dairy_SimpleAvg', 'Dairy_WeightedAvg', 'Meat_Imports_EuroM'] …


In [54]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

EXPORT_FILE = Path("/Users/gouthamjekkula/Desktop/Thesis Folder/Export_Master_Dataset_Clean.xlsx")
PPML_FILE   = Path("/Users/gouthamjekkula/Desktop/Thesis Folder/PPML_Gravity_Tariffs1.xlsx")
OUT_FILE    = Path("PPML_clean_Dataset1.xlsx")

 # 1) Export_Master (left table) — normalize Date and lock span
exp = pd.read_excel(EXPORT_FILE)
exp.columns = [c.strip() for c in exp.columns]
if "Date" not in exp.columns:
    raise ValueError("Export_Master_Dataset_Clean.xlsx must contain 'Date'.")

exp["Date"] = pd.to_datetime(exp["Date"], errors="coerce").dt.to_period("M").dt.to_timestamp()
date_min, date_max = exp["Date"].min(), exp["Date"].max()
exp = exp[(exp["Date"] >= date_min) & (exp["Date"] <= date_max)].copy()

# 2) PPML Gravity + Tariffs — normalize Date and trim to export span
ppml = pd.read_excel(PPML_FILE)
ppml.columns = [c.strip() for c in ppml.columns]
if "Date" not in ppml.columns:
    raise ValueError("PPML_Gravity_AHS_Tariffs.xlsx must contain 'Date'.")

ppml["Date"] = pd.to_datetime(ppml["Date"], errors="coerce").dt.to_period("M").dt.to_timestamp()
ppml = ppml[(ppml["Date"] >= date_min) & (ppml["Date"] <= date_max)].copy()

# 3) Identify tariff vs gravity columns in PPML
sector_prefixes = ("Pharma_", "Agriculture_", "Beverages_", "Dairy_", "Meat_", "Vegetables_")
tariff_cols = [c for c in ppml.columns if c.startswith(sector_prefixes)]

# gravity = everything else (except Date and the tariff cols)
gravity_cols = [c for c in ppml.columns if c not in (["Date"] + tariff_cols)]

ppml_keep = ppml[["Date"] + gravity_cols + tariff_cols].copy()

# 4) Merge on Date (left join keeps export’s exact monthly span)
merged = exp.merge(ppml_keep, on="Date", how="left", suffixes=("", "_ppmldup"))

# 5) Fill missing tariff columns with 0 (true pre‑Brexit & out-of-range months)
if tariff_cols:
    merged[tariff_cols] = merged[tariff_cols].fillna(0)

# 6) Create Euro‑Millions versions of export value columns (inserted right after each original)
#    These are in Euro Thousand in Export_Master
export_value_cols = [
    "Agriculture", "Beverages", "Dairy", "Meat", "Pharmaceuticals", "Vegetables", "Other",
    "6_Total_Exports_TSM09", "Total_Exports_All_commodities"
]
export_value_cols = [c for c in export_value_cols if c in merged.columns]

def insert_after(lst, after, new_item):
    out = []
    for x in lst:
        out.append(x)
        if x == after:
            out.append(new_item)
    return out

# Build column order with EuroM right after originals
col_order = merged.columns.tolist()
for col in export_value_cols:
    euro_col = f"{col}_EuroM"
    merged[euro_col] = pd.to_numeric(merged[col], errors="coerce") / 1000.0  # K€ -> €M
    if euro_col not in col_order:
        col_order = insert_after(col_order, col, euro_col)

# 7) (Optional polish) Group sector triples: Value, EuroM, (%) if present
sectors = ["Agriculture","Beverages","Dairy","Meat","Pharmaceuticals","Vegetables","Other"]
meta_cols = [c for c in ["Date","Statistic Label","Country","UNIT"] if c in merged.columns]

grouped_sector_cols = []
for s in sectors:
    if s in merged.columns:
        grouped_sector_cols.append(s)
    if f"{s}_EuroM" in merged.columns:
        grouped_sector_cols.append(f"{s}_EuroM")
    if f"{s} (%)" in merged.columns:
        grouped_sector_cols.append(f"{s} (%)")

# totals
grouped_totals = []
for t in ["6_Total_Exports_TSM09","Total_Exports_All_commodities"]:
    if t in merged.columns:
        grouped_totals.append(t)
    if f"{t}_EuroM" in merged.columns:
        grouped_totals.append(f"{t}_EuroM")

# Arrange tariff blocks
avg_cols    = sorted([c for c in tariff_cols if c.endswith("_WeightedAvg") or c.endswith("_SimpleAvg")])
import_cols = sorted([c for c in tariff_cols if c.endswith("_Imports_EuroM")])

# Gravity block: keep original order of gravity_cols
gravity_block = [c for c in gravity_cols if c in merged.columns]

# Final order (add any leftovers at the end)
ordered = []
ordered += meta_cols
ordered += grouped_sector_cols
ordered += grouped_totals
ordered += gravity_block
ordered += avg_cols
ordered += import_cols
leftovers = [c for c in merged.columns if c not in ordered]
ordered += leftovers

merged = merged[ordered]

# 8) Save
merged.to_excel(OUT_FILE, index=False)
print(f"[OK] Saved → {OUT_FILE}")
print(f"Rows: {len(merged):,} | Columns: {len(merged.columns)}")
print("Sample tariff cols:", (avg_cols + import_cols)[:10])
print("Sample gravity cols:", gravity_block[:10])
print("First 15 columns:", merged.columns[:15].tolist())


[OK] Saved → PPML_clean_Dataset1.xlsx
Rows: 125 | Columns: 67
Sample tariff cols: ['Agriculture_SimpleAvg', 'Agriculture_WeightedAvg', 'Beverages_SimpleAvg', 'Beverages_WeightedAvg', 'Dairy_SimpleAvg', 'Dairy_WeightedAvg', 'Meat_SimpleAvg', 'Meat_WeightedAvg', 'Pharma_SimpleAvg', 'Pharma_WeightedAvg']
Sample gravity cols: ['year', 'country_id_o', 'country_id_d', 'iso3_o', 'iso3_d', 'distw_harmonic', 'distw_arithmetic', 'dist', 'contig', 'comlang_off']
First 15 columns: ['Date', 'Statistic Label', 'Country', 'UNIT', 'Agriculture', 'Agriculture_EuroM', 'Agriculture (%)', 'Beverages', 'Beverages_EuroM', 'Beverages (%)', 'Dairy', 'Dairy_EuroM', 'Dairy (%)', 'Meat', 'Meat_EuroM']


In [56]:
import pandas as pd
from pathlib import Path

IN_FILE  = Path("/Users/gouthamjekkula/Desktop/Thesis Folder/PPML_clean_Dataset1.xlsx")
OUT_FILE = Path("PPML_clean_Dataset.xlsx")

# 1) Load 
df = pd.read_excel(IN_FILE)
df.columns = [c.strip() for c in df.columns]

#2) Parse date & time_id 
if "Date" not in df.columns:
    raise ValueError("Expected a 'Date' column.")
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df = df.dropna(subset=["Date"]).copy()
if "Year" not in df.columns:
    df["Year"] = df["Date"].dt.year
if "Month" not in df.columns:
    df["Month"] = df["Date"].dt.month
df["time_id"] = df["Date"].dt.to_period("M").astype(str)  # e.g., '2019-07'

# 3) Columns to DROP 
drop_cols = [
    "Statistic Label", "Country", "UNIT",  # redundant text columns
    "Other_EuroM"                          # out of thesis scope
]
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

#  4) Keep exactly what we need for PPML + Hybrid 
# Sectors (dependent variables)
sectors = ["Agriculture_EuroM","Beverages_EuroM","Dairy_EuroM",
           "Meat_EuroM","Pharmaceuticals_EuroM","Vegetables_EuroM"]

# Gravity covariates
gravity = [
    "GDP_IE_real_eur_m","GDP_UK_real_eur_m",
    "gdpcap_o","gdpcap_d","pop_o","pop_d",
    "distw_harmonic","contig","comlang_off","comcol","col45",
    "rta_coverage","rta_type","iso3_d"
]

# Tariffs (keep WeightedAvg + Imports_EuroM by sector)
tariff_weighted = [
    "Agriculture_WeightedAvg","Beverages_WeightedAvg","Dairy_WeightedAvg",
    "Meat_WeightedAvg","Pharmaceuticals_WeightedAvg","Vegetables_WeightedAvg"
]
tariff_imports = [
    "Agriculture_Imports_EuroM","Beverages_Imports_EuroM","Dairy_Imports_EuroM",
    "Meat_Imports_EuroM","Pharmaceuticals_Imports_EuroM","Vegetables_Imports_EuroM"
]

# Time identifiers
time_cols = ["Date","Year","Month","Period","time_id"]

# Build final keep list (only columns that actually exist to avoid KeyErrors)
final_keep = [c for c in (time_cols + sectors + gravity + tariff_weighted + tariff_imports) if c in df.columns]
df_final = df[final_keep].copy()

#sort for neatness
df_final = df_final.sort_values(["Date"]).reset_index(drop=True)

#5) Save modelling-ready dataset
df_final.to_excel(OUT_FILE, index=False)
print(f"[OK] Saved → {OUT_FILE}")
print("Columns kept:", len(df_final.columns))


[OK] Saved → PPML_clean_Dataset.xlsx
Columns kept: 34
